In [1]:
# Enable GPU before running

In [2]:
# clone the ADIS repository
!git clone https://github.com/sathishkumar67/ADIS.git
# move the files to the current directory
!mv /content/ADIS/* /content/
# upgrade pip
!pip install --upgrade pip
# install the required packages
!pip install -r requirements.txt

Cloning into 'ADIS'...
remote: Enumerating objects: 1253, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1253 (delta 39), reused 81 (delta 21), pack-reused 1154 (from 2)
Receiving objects: 100% (1253/1253), 40.21 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (632/632), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# necessary imports
from __future__ import annotations
import os
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# LOCAL DIRECTORY
LOCAL_DIR = os.getcwd()

# HUGGINGFACE
REPO_ID = "pt-sk/ADIS"
DATASET_NAME = "balanced_dataset"
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
REPO_TYPE = "dataset"
MODEL_REPO_TYPE = "model"

# DATA PATHS
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"

# DATASET
NUM_CLASSES = 10
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake']
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"

In [5]:
# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

# remove dataset.zip
os.remove(DATASET_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


balanced_dataset.zip:   0%|          | 0.00/7.04G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 7.07G/7.07G [01:16<00:00, 92.0MB/s]


In [9]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)

## Define which model do you want to evaluate with

### Unoptimized Models:
  1. yolov11n_without_bohb_tuned_ckpt.pt
  2. yolov11s_without_bohb_tuned_ckpt.pt
  3. yolov11m_without_bohb_tuned_ckpt.pt

### Optimized Models:
  1. yolov11n_bohb_tuned_ckpt.pt
  2. yolov11s_bohb_tuned_ckpt.pt
  3. yolov11m_bohb_tuned_ckpt.pt

Enter the model name to download the saved model from huggingface hub


In [14]:
MODEL_NAME = "yolov11n_bohb_tuned_ckpt.pt"

# download the model
hf_hub_download(repo_id=REPO_ID, filename=MODEL_NAME, repo_type=MODEL_REPO_TYPE, local_dir=LOCAL_DIR)

# load the model with the pretrained weights
model = YOLO11Model(MODEL_NAME, verbose=True)

## Model Evaluation
To evaluate the model specify the split that you want to evaluate on. The options are:
  1. train
  2. val
  3. test


In [18]:
# evaluate the model on various splits
SPLIT = "val"
import time
start_time = time.time()
model.val(data=DATA_YAML_FILE, split=SPLIT)
print(f"Time taken to evaluate on {SPLIT} split: {time.time() - start_time} seconds")

Ultralytics 8.3.98 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 164 layers, 2,905,382 parameters, 0 gradients, 7.6 GFLOPs


val: Scanning /content/balanced_dataset/val.cache... 2390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2390/2390 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:26<00:00,  5.69it/s]


                   all       2390       3083      0.887      0.796      0.886      0.642
          Images  Instances  Precision  Recall  mAP50  mAP50-95  F1-Score  IoU   Accuracy
Cat       239     262       0.933      0.900   0.954  0.736     0.916     0.875 0.875    
Cattle    239     351       0.877      0.672   0.839  0.592     0.761     0.851 0.693    
Chicken   239     495       0.881      0.669   0.803  0.553     0.760     0.847 0.726    
Deer      239     326       0.895      0.847   0.914  0.698     0.870     0.873 0.808    
Dog       239     272       0.874      0.724   0.847  0.542     0.792     0.815 0.709    
Squirrel  239     245       0.874      0.824   0.890  0.638     0.848     0.856 0.807    
Eagle     239     252       0.924      0.869   0.949  0.805     0.896     0.909 0.835    
Goat      239     332       0.823      0.755   0.829  0.606     0.787     0.865 0.706    
Rodents   239     280       0.895      0.832   0.914  0.650     0.863     0.856 0.817    
Snake     2